# **Sistem Rekomendasi Buku Berdasarkan Preferensi dan Rating Pengguna**

Oleh: Rizki Alvian

## **Install dan import library yang dibutuhkan**

In [1]:
# Data preparation dan visualization
import numpy as np
import pandas as pd
import gzip
import seaborn as sns
import re
import random
import zipfile, os
import warnings
warnings.filterwarnings("ignore")

# Membangun dan analisa sistem rekomendasi
!pip install surprise

# Membangun model dan evaluasi
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise import accuracy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3096337 sha256=c317f62f5f4a380eb5e69fd901a6d34d950b2e3e8cb6267ba671b31e6680e941
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


## **Menyiapkan data yang akan digunakan**

Mengambil data dari kaggle.

Link: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset

In [2]:
# Instal library kaggle
! pip install kaggle

In [3]:
# Membuat folder .kaggle di dalam folder root
!rm -rf ~/.kaggle && mkdir ~/.kaggle/

In [4]:
# Menyalin berkas kaggle.json pada direktori aktif saat ini ke folder .kaggle
!mv kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# Unduh dataset menggunakan Kaggle CLI
!kaggle datasets download -d arashnic/book-recommendation-dataset

 21% 5.00M/23.8M [00:00<00:00, 30.0MB/s]
100% 23.8M/23.8M [00:00<00:00, 98.8MB/s]


In [6]:
# Ekstraksi berkas zip ke direktori aktif saat ini
!unzip /content/book-recommendation-dataset.zip

Archive:  /content/book-recommendation-dataset.zip
  inflating: Books.csv               
  inflating: Ratings.csv             
  inflating: Users.csv               
  inflating: recsys_taxonomy2.png    


## **Exploratory Data Analysis (EDA)**

In [7]:
df_book = pd.read_csv('Books.csv')
df_rating = pd.read_csv('Ratings.csv')
df_user = pd.read_csv('Users.csv')

Melihat karakteristik data buku

In [8]:
df_book

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [9]:
df_book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [10]:
print('Total ISBN adalah:', len(df_book.ISBN.unique()))
print('Total judul adalah:', len(df_book['Book-Title'].unique()))
print('Total pengarang adalah:', len(df_book['Book-Author'].unique()))
print('Total penerbit adalah:', len(df_book['Publisher'].unique()))

Total ISBN adalah: 271360
Total judul adalah: 242135
Total pengarang adalah: 102024
Total penerbit adalah: 16808


Melihat karakteristik data rating

In [11]:
df_rating

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [12]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [13]:
print('Total pengguna adalah:', len(df_rating['User-ID'].unique()))
print('Total buku adalah:', len(df_rating['ISBN'].unique()))
print('Total rating yang diterima adalah:', len(df_rating))

Total pengguna adalah: 105283
Total buku adalah: 340556
Total rating yang diterima adalah: 1149780


Melihat karakteristik data user

In [14]:
df_user

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [15]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


## **Data Preparation**

Menyiapkan fitur-fitur yang diperlukan pada data buku

In [16]:
buku = df_book[['ISBN', 'Book-Title','Book-Author', 'Publisher']]
buku.head()

,ISBN,Book-Title,Book-Author,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company


In [17]:
# memeriksa missing value
buku.isnull().sum()

ISBN           0
Book-Title     0
Book-Author    1
Publisher      2
dtype: int64

In [18]:
# mengisi missing value pada publisher dan author
buku.loc[:,'Book-Author'] = buku['Book-Author'].fillna('Unknown')
buku.loc[:,'Publisher'] = buku['Publisher'].fillna('Unknown')
buku.isnull().sum()

ISBN           0
Book-Title     0
Book-Author    0
Publisher      0
dtype: int64

Menyiapkan fitur-fitur yang diperlukan pada data rating

In [19]:
rating = df_rating
rating.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [20]:
print('Jumlah rating 0 :', rating['Book-Rating'].eq(0).sum())
rating.shape

Jumlah rating 0 : 716109


(1149780, 3)

In [21]:
# menghapus data rating dengan nilai 0
rating = rating[df_rating['Book-Rating']>0]
rating.shape

(433671, 3)

Menyiapkan fitur-fitur yang diperlukan pada data user

In [22]:
user = df_user
user.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [23]:
# mengisi missing value dengan moduse dari Age
user['Age'] = user['Age'].fillna(user['Age'].mode())
user.isnull().sum()

User-ID          0
Location         0
Age         110761
dtype: int64

In [24]:
# menggabungkan data buku dan rating
rating_buku = pd.merge(rating, buku, on=['ISBN'],)
rating_buku.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Publisher
0,276726,0155061224,5,Rites of Passage,Judith Rae,Heinle
1,276729,052165615X,3,Help!: Level 1,Philip Prowse,Cambridge University Press
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,Cambridge University Press
3,276744,038550120X,7,A Painted House,JOHN GRISHAM,Doubleday
4,11676,038550120X,10,A Painted House,JOHN GRISHAM,Doubleday


## **Modeling**

**Popularity based recommendation**

Tahapan-tahapan berikut akan menghasilkan rekomendasi buku paling populer berdasarkan rata-rata dan jumlah rating.

In [25]:
# Berdasarkan rata-rata rating
rata_rating_buku = rating_buku.groupby('ISBN')['Book-Rating'].mean().sort_values(ascending=False)

# Rekomendasi dari rating yang diterima
buku_populer = rating_buku.groupby('ISBN')['Book-Rating'].count().sort_values(ascending=False)

**Weighted rating**

Digunakan untuk menggabungkan kedua informasi lalu kemudian akan dipilih 10 top buku

In [26]:
rekomendasi_populer = pd.concat([rata_rating_buku, buku_populer],
                                   axis=1, join='inner',
                                   keys=['Average Rating', 'Rating Received'])

In [27]:
v = rekomendasi_populer['Rating Received']
R = rekomendasi_populer['Average Rating']
C = rekomendasi_populer['Average Rating'].mean()
m = rekomendasi_populer['Rating Received'].quantile(0.75)

rekomendasi_populer['Weighted Rating'] = ((R*v)+(C*m))/(v+m)
rekomendasi_populer = rekomendasi_populer.sort_values('Weighted Rating', ascending=False)
rekomendasi_populer.head(10)

,Average Rating,Rating Received,Weighted Rating
ISBN,,,
0439425220,9.869565,23,9.682203
1888054557,10.000000,11,9.619622
0836213319,9.923077,13,9.603672
0618002235,9.720000,25,9.557596
0060256656,9.750000,20,9.547958
0394800893,10.000000,8,9.505509
0394800389,9.785714,14,9.503443
1571456988,10.000000,7,9.450565
089471838X,10.000000,7,9.450565


In [28]:
# hasil rekomendasi
judul_populer = pd.merge(rekomendasi_populer, df_book[['ISBN', 'Book-Title']], on='ISBN')
judul_populer = judul_populer[['Weighted Rating', 'Book-Title']]
judul_populer = judul_populer.drop_duplicates('Book-Title').set_index('Book-Title')
judul_populer.head(10)

,Weighted Rating
Book-Title,
Harry Potter and the Chamber of Secrets Postcard Book,9.682203
Postmarked Yesteryear: 30 Rare Holiday Postcards,9.619622
Dilbert: A Book of Postcards,9.603672
"The Two Towers (The Lord of the Rings, Part 2)",9.557596
The Giving Tree,9.547958
The Sneetches and Other Stories,9.505509
Fox in Socks (I Can Read It All by Myself Beginner Books),9.503443
Uncle John's Supremely Satisfying Bathroom Reader (Uncle John's Bathroom Reader),9.450565
Natural California: A Postcard Book,9.450565


**Model-Based Collaborative filtering Recommendation**

Tahapan-tahapan berikut akan menghasilkan pemberian rekomendasi buku lain kepada user setelah memberi rating.

In [29]:
rating_user = rating_buku
rating_user

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Publisher
0,276726,0155061224,5,Rites of Passage,Judith Rae,Heinle
1,276729,052165615X,3,Help!: Level 1,Philip Prowse,Cambridge University Press
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,Cambridge University Press
3,276744,038550120X,7,A Painted House,JOHN GRISHAM,Doubleday
4,11676,038550120X,10,A Painted House,JOHN GRISHAM,Doubleday
...,...,...,...,...,...,...
383837,276688,0425156737,2,Graven Images,Jane Waterhouse,Berkley Publishing Group
383838,276688,0671563149,6,MUDDY WATER (Peter Bartholomew Mysteries),Sally Gunning,Pocket
383839,276688,1575660792,7,Gray Matter,Shirley Kennett,Kensington Publishing Corporation
383840,276704,0380796155,5,White Abacus,Damien Broderick,Eos


In [30]:
# encoder User ID dan Book Title
le = LabelEncoder()
rating_user['UserID'] = le.fit_transform(rating_user['User-ID'])
rating_user['TitleID'] = le.fit_transform(rating_user['Book-Title'])
rating_user.drop('User-ID', axis=1, inplace=True)
rating_user

,ISBN,Book-Rating,Book-Title,Book-Author,Publisher,UserID,TitleID
0,0155061224,5,Rites of Passage,Judith Rae,Heinle,67541,84140
1,052165615X,3,Help!: Level 1,Philip Prowse,Cambridge University Press,67542,46362
2,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,Cambridge University Press,67542,97833
3,038550120X,7,A Painted House,JOHN GRISHAM,Doubleday,67543,3064
4,038550120X,10,A Painted House,JOHN GRISHAM,Doubleday,2719,3064
...,...,...,...,...,...,...,...
383837,0425156737,2,Graven Images,Jane Waterhouse,Berkley Publishing Group,67537,43523
383838,0671563149,6,MUDDY WATER (Peter Bartholomew Mysteries),Sally Gunning,Pocket,67537,63876
383839,1575660792,7,Gray Matter,Shirley Kennett,Kensington Publishing Corporation,67537,43547
383840,0380796155,5,White Abacus,Damien Broderick,Eos,67538,131513


**Pengembangan Model**

Melakukan training pada data menggunakan model SVD dari library surprise, lalu melakukan evaluasi dengan 10-fold cross validation menggunakan RMSE dan MAE.

Melakukan validasi silang 10 kali lipat, melakukan prosedur pemasangan sebanyak sepuluh kali, dengan masing-masing pemasangan dilakukan pada set pelatihan yang terdiri dari 90% set pelatihan dan 10% sebagai set validasi yang dipilih secara acak.

10 fold CV adalah salah satu K fold CV yang direkomendasikan untuk pemilihan model terbaik karena cenderung memberikan estimasi akurasi yang kurang bias dibandingkan dengan CV biasa.

In [31]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(rating_user[['UserID', 'TitleID', 'Book-Rating']], reader)
svd = SVD(verbose=False, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.6437  1.6329  1.6306  1.6323  1.6518  1.6362  1.6363  1.6377  1.6405  1.6433  1.6385  0.0061  
MAE (testset)     1.2696  1.2640  1.2613  1.2642  1.2746  1.2644  1.2664  1.2672  1.2693  1.2719  1.2673  0.0039  
Fit time          9.66    4.58    4.34    4.81    4.36    4.83    4.33    4.95    4.36    5.05    5.13    1.53    
Test time         0.45    0.45    0.55    0.46    0.53    0.79    0.26    0.63    0.28    0.32    0.47    0.15    


{'test_rmse': array([1.64369058, 1.63287864, 1.6305752 , 1.63234223, 1.6517868 ,
        1.6361703 , 1.63630832, 1.63768626, 1.64050467, 1.6433383 ]),
 'test_mae': array([1.26964053, 1.26398184, 1.26128573, 1.26416228, 1.27456482,
        1.26437673, 1.26642557, 1.26718528, 1.26934377, 1.27188269]),
 'fit_time': (9.663792610168457,
  4.581392526626587,
  4.341421842575073,
  4.814676284790039,
  4.3598291873931885,
  4.829602479934692,
  4.3326475620269775,
  4.948465585708618,
  4.360919713973999,
  5.053489923477173),
 'test_time': (0.4503612518310547,
  0.4534292221069336,
  0.5529928207397461,
  0.46382594108581543,
  0.5311214923858643,
  0.7888290882110596,
  0.26442480087280273,
  0.6259684562683105,
  0.28203821182250977,
  0.31707191467285156)}

## **Memberi rekomendasi berdasarkan riwayat rating**

In [32]:
def user_recommendation(userid):
    user = rating_user[['ISBN', 'Book-Title', 'Book-Author', 'Publisher', 'TitleID']]
    user = user.reset_index()

    # mendapatkan dataset lengkap
    data = Dataset.load_from_df(rating_user[['UserID','TitleID','Book-Rating']], reader)
    trainset = data.build_full_trainset()
    svd.fit(trainset)
    user['Estimate_Score'] = user['TitleID'].apply(lambda x: svd.predict(userid, x).est)
    user = user.drop(['index','TitleID'], axis = 1)
    user = user.sort_values('Estimate_Score' , ascending = False)
    counts1 = user['Estimate_Score'].value_counts()
    user = user[user['Estimate_Score'].isin(counts1[counts1 == 1].index)]
    return user.head(10)

In [33]:
# mencari user yang menyukai buku Harry Potter
rating_user[rating_user['Book-Title'].str.contains('Harry Potter')]

,ISBN,Book-Rating,Book-Title,Book-Author,Publisher,UserID,TitleID
367,043935806X,7,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,Scholastic,67553,45385
368,043935806X,10,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,Scholastic,67966,45385
369,043935806X,9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,Scholastic,481,45385
370,043935806X,9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,Scholastic,1461,45385
371,043935806X,10,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,Scholastic,1511,45385
...,...,...,...,...,...,...,...
356778,043932162X,8,Harry Potter Schoolbooks Box Set: Two Classic ...,J. K. Rowling,Scholastic,53159,45364
356779,043932162X,10,Harry Potter Schoolbooks Box Set: Two Classic ...,J. K. Rowling,Scholastic,65422,45364
373343,0972322108,10,The Hidden Key to Harry Potter: Understanding ...,John Granger,Zossima Press,60037,107251
373626,043935806x,10,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,Scholastic,60216,45385


In [34]:
user_id = 10089
print('Rating buku dari user :')
rating_user[rating_user['UserID'].eq(user_id)]

Rating buku dari user :


,ISBN,Book-Rating,Book-Title,Book-Author,Publisher,UserID,TitleID
27341,0385498802,8,Bee Season: A Novel,Myla Goldberg,Anchor Books/Doubleday,10089,12078
44787,2253140872,9,L'Ecume Des Jours,Boris Vian,Distribooks Inc,10089,56579
51924,2070375161,5,La Ferme Des Animaux,George Orwell,Gallimard French,10089,57282
89670,0061073628,9,A Night Without Armor : Poems,Jewel,Perennial Currents,10089,3036
167376,0140481346,7,Death of A Salesman,Arthur Miller,Penguin USA (Paper),10089,26496
225892,0060512377,7,Does She or Doesn't She?,Alisa Kwitney,Avon Trade,10089,30136
225896,0061058262,7,"The Crow: Clash by Night (The Crow, No 3)",Chet Williamson,Perennial Currents,10089,102531
225898,0345417127,10,The Crow: Shattered Lives &amp; Broken Dreams,J. O'Barr,Del Rey Books,10089,102533
225899,0425157415,8,"Vampires, Wine and Roses",John Richard Stephens,Berkley Publishing Group,10089,127583
225900,225303598X,8,L'histoire sans fin,Ende M.,LGF,10089,56736


In [35]:
print('Buku yang mungkin disukai user')
user_recommendation(user_id)

Buku yang mungkin disukai user


,ISBN,Book-Title,Book-Author,Publisher,Estimate_Score
225898,0345417127,The Crow: Shattered Lives &amp; Broken Dreams,J. O'Barr,Del Rey Books,8.151110
216493,0835911071,Star Trek Next Generation: Contamination,John Vornholt,Globe Fearon,8.113014
262682,3596148642,Austerlitz.,Winfried G. Sebald,"Fischer (Tb.), Frankfurt",8.100955
235769,8478887121,LA Nochevieja De Montalbano (Narrativa),Andrea Camilleri,Salamandra,8.077965
320586,0553443410,"A Prince for Jenny (Loveswept, No 656)",Peggy Webb,Loveswept,8.074810
337287,0373033346,"Sensible Wife (Harlequin Romance, No 3334)",Jessica Hart,Harlequin,8.068090
375945,0451114876,The Mystery of the Black Diamonds,Phyllis A. Whitney,New Amer Library (Mm),8.062611
191801,1565652061,The Diabetes Sports and Exercise Book: How to ...,Claudia Graham,McGraw-Hill,8.058640
320485,0373691467,Heatwave,Jamie Denton,Harlequin,8.051578
332422,0393307921,Arrogance,Joanna Scott,Simon &amp; Schuster,8.051044
